# Стекинг

Стекинг (stacking) — агрегация ответов моделей машинного обучения при помощи ещё одной модели машинного обучения. Подход использует понятие базовых моделей, каждая из которых обучается независимо от остальных, и мета-модели, которая использует предсказания базовых моделей как признаки.

Правила стекинга:

- Обучать мета-модели нужно на новых для базовых моделей данных. Так избегается data leak, появляющийся при оценке алгоритмов на обучающей выборке.
- Можно использовать подходы из кросс-валидации (разбиение по фолдам, бутстрап) для получения различных моделей.
- Лучше применять регрессоры для мета-моделей.
- Для старта лучше взять простую мета-модель, например, линейную регрессию. В случае классификации — для агрегации вероятности классов. Этот частный случай называют блендингом (blending).
- Брать в качестве базовых моделей модели различной природы.
- Обучать модель не только на истинный таргет, но и на прокси таргет, например, на важный признак, который выявили во время исследования данных.
- Делать стекинг с большим количеством уровней при наличии достаточного количества данных, времени и усилий, что на практике редко.

## Вопросы:
- Как необходимо настраивать гиперпараметры для базовых алгоритмов в стекинге
    - На отложенной выборке


- На что влияет увеличение количество фолдов в стекинге?
    - Увеличивается время обучения
    - Увеличивается качество итоговой модели


- Как применять стекинг для предсказаний (inference) на совершенно новых объектах?
    - Сохранить все обученные на фолдах промежуточные модели и применить их на новых объектах с усреднением ответов


- Какие из следующих утверждений верны для стекинга?
    - Модель обучается на предсказаниях разных моделей машинного обучения
    - Модели на первой стадии обучаются на признаках исходного набора данных


- Какие из перечисленных ниже действий могут выполняться при стекинге?
    - Разделить обучающую выборку на k фолдов
    - Обучить k моделей на каждом из k-1 фолдов и получить предсказания для оставшегося фолда


- Вы используете стекинг с n методами машинного обучения и данными, разбитыми на k фолдов, для решения задачи бинарной классификации. Все базовые модели обучаются на всех признаках исходного датасета. Вы используете k фолдов для базовых моделей. Какое утверждение верно для стекинга с одним уровнем (m базовых моделей и одна агрегирующая результаты модель)?
    - После первой стадии у вас останется только m признаков

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [4]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):

    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(
            X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [5]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):

    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]

    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])

    return stacked_features_train, stacked_features_test

In [6]:
cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)


def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

## Задание 1

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
- логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
- случайный лес из 300 деревьев
- градиентный бустинг из 200 деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [7]:
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(penalty='l1', C=0.001, solver='saga',
                       multi_class='ovr', max_iter=2000, random_state=RANDOM_STATE),
    LogisticRegression(penalty='l2', C=0.001, solver='saga',
                       multi_class='multinomial', max_iter=2000, random_state=RANDOM_STATE),
    RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE),
    GradientBoostingClassifier(n_estimators=200, random_state=RANDOM_STATE)
], X_train, X_test, y_train, cv)

100%|██████████| 4/4 [03:23<00:00, 50.86s/it]


In [8]:
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs')

In [9]:
compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

0.98339

## Задание 2

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- случайный лес из 300 деревьев
- случайный лес из 200 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [10]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE),
    ExtraTreesClassifier(n_estimators=200, random_state=RANDOM_STATE)
], X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:16<00:00,  8.28s/it]


In [11]:
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs')

In [12]:
compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

0.981296

## Задание 3
Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- метод ближайшего соседа (k-NN) со стандартными параметрами
- случайный лес из 300 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [13]:
stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=200, random_state=RANDOM_STATE)
], X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:06<00:00,  3.39s/it]


In [14]:
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs')

In [15]:
compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

0.987798

## Задание 4

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
- метод ближайшего соседа со стандартными параметрами
- случайный лес из 300 экстремальных деревьев
- AdaBoost со стандартными параметрами

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [16]:
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(penalty='l1', C=0.001, solver='saga',
                       multi_class='ovr', max_iter=2000, random_state=RANDOM_STATE),
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE),
    AdaBoostClassifier(random_state=RANDOM_STATE)
], X_train, X_test, y_train, cv)

100%|██████████| 4/4 [01:03<00:00, 15.87s/it]


In [17]:
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs')

In [18]:
compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

0.987798

## Задание 5

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- случайный лес из 300 деревьев
- случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для корректной работы необходимо подправить код в функции compute_meta_feature. Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. Количество фолдов = 10

In [19]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):

    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train, y_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(
            X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [20]:
scv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE),
    ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE)
], X_train, X_test, y_train, scv)

log_reg = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=RANDOM_STATE)

compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

100%|██████████| 2/2 [00:18<00:00,  9.37s/it]


0.981296

В предыдущей задаче измените 10 фолдов на 20. Укажите полученное качество.

In [21]:
scv = StratifiedKFold(n_splits=20, shuffle=True, random_state=RANDOM_STATE)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE),
    ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE)
], X_train, X_test, y_train, scv)

log_reg = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=RANDOM_STATE)

compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

100%|██████████| 2/2 [00:34<00:00, 17.07s/it]


0.981296

В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [22]:
scv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(random_state=RANDOM_STATE),
    ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE)
], X_train, X_test, y_train, scv)

log_reg = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=RANDOM_STATE)

compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

100%|██████████| 2/2 [00:05<00:00,  2.82s/it]


0.978682

В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [23]:
scv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE)
], X_train, X_test, y_train, scv)

log_reg = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=RANDOM_STATE)

compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

100%|██████████| 2/2 [00:04<00:00,  2.20s/it]


0.987798

В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [24]:
scv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

stacked_features_train, stacked_features_test = generate_meta_features([
    GradientBoostingClassifier(random_state=RANDOM_STATE),
    ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE)
], X_train, X_test, y_train, scv)

log_reg = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=RANDOM_STATE)

compute_metric(log_reg, stacked_features_train, y_train, stacked_features_test)

100%|██████████| 2/2 [00:41<00:00, 20.81s/it]


0.980205

## Задание 6

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
- случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
Для генерации фолдов используйте класс StratifiedKFold и поправленный Вами ранее код в функции compute_meta_feature.

Выполните разбиение на 3 фолда.

Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [25]:
scv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(max_depth=24, criterion='gini', random_state=RANDOM_STATE),
    ExtraTreesClassifier(n_estimators=300, random_state=RANDOM_STATE)
], X_train, X_test, y_train, scv)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)

compute_metric(rf_clf, stacked_features_train, y_train, stacked_features_test)

100%|██████████| 2/2 [00:03<00:00,  1.93s/it]


0.981376

## Задание 7

Обучите на тренировочной выборке следующие алгоритмы:

- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
- случайный лес из 300 экстремальных деревьев
- логистическую регрессию со стандартными параметрами

Усредните их ответы на тестовой выборке методом сложения предсказаний и затем взятия функции argmax: answer = (prediction1 + prediction2 + prediction3).argmax(axis = 1).

Посчитайте качество, аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

In [26]:
def compute_predicts(classifiers, X_train, y_train, X_test):
    predicts = []
    for clf in classifiers:
        clf.fit(X_train, y_train)
        predicts.append(clf.predict_proba(X_test))
    return predicts

In [27]:
predicts = compute_predicts([
    RandomForestClassifier(max_depth=24, criterion='gini', n_jobs=-1, random_state=RANDOM_STATE),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=RANDOM_STATE),
    LogisticRegression(random_state=RANDOM_STATE)
], X_train, y_train, X_test)

/home/eldar/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
y_test_pred = np.sum(predicts, axis=0).argmax(axis=1)
print(np.round(f1_score(y_test, y_test_pred, average='macro'), 6))

0.976259
